<a href="https://colab.research.google.com/github/HelenNunez/BALG_Project3/blob/aranza3/Training_Clustering_Test_4_LessPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

In [ ]:
pip install tensorflow

In [ ]:
pip install --upgrade tensorflow-datasets

Requirement already up-to-date: tensorflow-datasets in /usr/local/lib/python3.6/dist-packages (3.2.1)


In [ ]:
pip install -q tensorflow>=2 tfds-nightly matplotlib

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds 
import pandas as pd

In [ ]:
# Loading the wikipedia dataset.
DATASET_NAME = 'wikipedia/20190301.en'
# DATASET_NAME = 'wikipedia/20190301.uk'

dataset, dataset_info = tfds.load(
    name=DATASET_NAME,
    data_dir='tmp',
    with_info=True,
    split=tfds.Split.TRAIN,
)

In [ ]:
print(dataset_info)

tfds.core.DatasetInfo(
    name='wikipedia',
    version=1.0.0,
    description='Wikipedia dataset containing cleaned articles of all languages.
The datasets are built from the Wikipedia dump
(https://dumps.wikimedia.org/) with one split per language. Each example
contains the content of one full Wikipedia article with cleaning to strip
markdown and unwanted sections (references, etc.).',
    homepage='https://dumps.wikimedia.org',
    features=FeaturesDict({
        'text': Text(shape=(), dtype=tf.string),
        'title': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=5824596,
    splits={
        'train': 5824596,
    },
    supervised_keys=None,
    citation="""@ONLINE {wikidump,
        author = "Wikimedia Foundation",
        title  = "Wikimedia Downloads",
        url    = "https://dumps.wikimedia.org"
    }""",
    redistribution_info=license: "This work is licensed under the Creative Commons Attribution-ShareAlike 3.0 Unported License. To view a copy of this l

In [ ]:
print(dataset)

<PrefetchDataset shapes: {text: (), title: ()}, types: {text: tf.string, title: tf.string}>


In [ ]:
#Find how many train examples we have available
TRAIN_NUM_EXAMPLES = dataset_info.splits['train'].num_examples
print('Total number of articles: ', TRAIN_NUM_EXAMPLES)

Total number of articles:  5824596


In [ ]:
#Print first article to view as an example
print('First article','\n======\n')
for example in dataset.take(1):
    print('Title:','\n------')
    print(example['title'].numpy().decode('utf-8'))
    print()

    print('Text:', '\n------')
    print(example['text'].numpy().decode('utf-8'))

First article 

Title: 
------
Joseph Greenberg

Text: 
------
Joseph Harold Greenberg (May 28, 1915 – May 7, 2001) was an American linguist, known mainly for his work concerning linguistic typology and the genetic classification of languages.

Life

Early life and education 
(Main source: Croft 2003)

Joseph Greenberg was born on May 28, 1915 to Jewish parents in Brooklyn, New York. His first great interest was music. At the age of 14, he gave a piano concert in Steinway Hall. He continued to play the piano frequently throughout his life.

After finishing high school, he decided to pursue a scholarly career rather than a musical one. He enrolled at Columbia University in New York. During his senior year, he attended a class taught by Franz Boas concerning American Indian languages. With references from Boas and Ruth Benedict, he was accepted as a graduate student by Melville J. Herskovits at Northwestern University in Chicago. During the course of his graduate studies, Greenberg did f

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [ ]:
word_rdd = spark.sparkContext.emptyRDD()
word_list = []
stop_words = set(stopwords.words("english"))
punctuations = [".", ",", "!", "?", ";", "#", ":", "(", ")", "-", "–" "/", "_","http", "https", "www", "\\", "@", "&", "[", "]","~","(",")"]

for article in dataset.take(100):
    sentences = nltk.sent_tokenize(article['text'].numpy().decode('utf-8'))
    for sentence in sentences:
      words = nltk.word_tokenize(sentence)
      without_stop_words = [word for word in words if word not in stop_words]
      clean_words = [[word for word in without_stop_words if word not in punctuations]]
      temp_rdd = spark.sparkContext.parallelize(clean_words)
      word_rdd = word_rdd.union(temp_rdd)

In [ ]:
#word_rdd

In [ ]:
#word_rdd.collect()

In [ ]:
# # Note this is an RDD approach, not dataframe. API is slightly different for dataframe
from pyspark.mllib.feature import Word2Vec
word2vec = Word2Vec()
model = word2vec.fit(word_rdd)

KeyboardInterrupt: ignored

In [ ]:
synonyms = model.findSynonyms('life', 5)

for word, cosine_distance in synonyms:
    print("{}: {}".format(word, cosine_distance))

In [ ]:
def cosine_similarity(x, y):
     return x.dot(y)/(x.norm(2)*y.norm(2))

In [ ]:
x = model.transform('life')
y = model.transform('Linguistics')
cosine_similarity(x, y)

In [ ]:
# synonyms = model.findSynonyms('diner', 2)

# for word, cosine_distance in synonyms:
#     print("{}: {}".format(word, cosine_distance))

In [ ]:
# def cosine_similarity(x, y):
#     return x.dot(y)/(x.norm(2)*y.norm(2))

# x = model.transform('diner')
# y = model.transform('slice,')
# cosine_similarity(x, y)